In [1]:
import alpaca_trade_api as tradeapi
import config
import logging
from alpaca_trade_api.rest import REST, TimeFrame
import datetime as dt
import pandas as pd
from datetime import date
import datetime
import yfinance as yf


#ally api endpoint creation
api = tradeapi.REST(key_id=config.LIVE_API_KEY, secret_key=config.LIVE_SECRET_KEY, api_version='v2',
                                        base_url=config.LIVE_API_BASE_URL)

positions = api.list_positions() # get account positions


#set start and end dates automatically
now = date.today()
today = now.strftime('%Y-%m-%d')
yesterday = (now - pd.Timedelta('1day')).strftime('%Y-%m-%d')
ninedaysago = (now - pd.Timedelta('8W')).strftime('%Y-%m-%d')


i = "TSLA"

#calculate percentage change of a stock's prrice between two periods in time. In this case,
#this prints the 2 month change of a given stock ticker.

def percent_change():
    quote_iter = api.get_bars(i, TimeFrame.Day, start = ninedaysago, end = yesterday, limit=9)._raw
    quote_df = pd.DataFrame(quote_iter)
    ninedago = quote_df.iloc[0]['c']
    pricenow = quote_df.iloc[8]['c']
    percent_change = (pricenow - ninedago) / ninedago * 100
    print(i, ' moved {:.2f}% over the last 2 months'.format(percent_change))
   


bool_long = bool
bool_short = bool
short_df = pd.DataFrame()
long_df = pd.DataFrame()


# create dataframe adding EMA columns to date, stock,     




def dataframe_creator():
    target_symbols_long = []
    target_symbols_short = []
    quote_iter = api.get_bars(i, TimeFrame.Day, start = ninedaysago, end = yesterday, limit=80)._raw
    quotes_df0 = pd.DataFrame(quote_iter)
    quotes_df = pd.DataFrame(index=None)
    quotes_df['Date'] = pd.to_datetime(quotes_df0['t'])
    quotes_df['Date'] = quotes_df['Date'].dt.strftime('%m/%d/%Y')
    quotes_df['Symbol'] = i
    quotes_df['Close'] = quotes_df0['c']
    quotes_df["EMA9"] = quotes_df0['c'].ewm(span=9).mean().round(2)
    quotes_df["EMA20"] = quotes_df0['c'].ewm(span=20).mean().round(2)
    quotes_df["EMA50"] =quotes_df0['c'].ewm(span=50).mean().round(2)
    quotes_df['Volume'] = quotes_df0['v'].apply(lambda x : "{:,}".format(x))
    bool_long = quotes_df["EMA9"].iloc[-1] < quotes_df["Close"].iloc[-1]  ##create Boolean to filter out stocks to go long on.
    if bool_long == True:
        target_list_long = i
        target_symbols_long.append(target_list_long)
        print("Stock ", i, "is within buying parameters. Dataframe: " , "\n")
        print(quotes_df.tail(1))
    else:
        print("Stock", i, "is not within buying parameters")

    bool_short = quotes_df["EMA9"].iloc[-1] > quotes_df["Close"].iloc[-1]  ##create second boolean to filter out stocks to short.
    if bool_short == True:
        target_list_short = i
        target_symbols_short.append(target_list_short)
        print("Stock ", i, "is within selling parameters. Dataframe: " , "\n")
        print(quotes_df.tail(1))
    else:
        print("Stock", i, "is not within selling parameters")
    
    
#more work needs to be done on slicing into the optimal option chain rows. 
#Example: for stock i find options where price is <.5. How to handle multiple results?


    
def option_chain_generator():
    chain_symbol = yf.Ticker(i)
    chain = chain_symbol.option_chain("2023-01-20")
    call_chain = chain.calls
    put_chain = chain.puts 
    if bool_long == True:
        try:
            optc = chain_symbol.option_chain("2023-01-20")
            optcz = optc.calls[['strike', 'lastPrice', 'bid', 'ask', 'volume', 'openInterest', 'impliedVolatility']][2:16]
            optczdf = pd.DataFrame(optcz)
            long_df.join(optczdf) 
            optczdf['lastTradeDate'] = optczdf['lastTradeDate'].dt.strftime('%m/%d/%Y')
            optczdf.rename(columns={"impliedVolatility": "IV", "openInterest": "OI"})
            print("\n", "BUY", i, ", Optimal Call Chain:", "\n\n", optczdf, "\n\n")
        except ValueError as err:
            print("no calls available for ", i, "\n\n")

    else:
        try:
            optp = chain_symbol.option_chain("2023-01-20")
            optpz = optp.puts[['lastTradeDate', 'strike', 'lastPrice', 'bid', 'ask', 'volume', 'openInterest', 'impliedVolatility']][20:50]
            optpzdf = pd.DataFrame(optpz)
            short_df.join(optpzdf)
            optpzdf['lastTradeDate'] = optpzdf['lastTradeDate'].dt.strftime('%m/%d/%Y')
            optpzdf.rename(columns={"impliedVolatility": "IV", "openInterest": "OI"})
            print("\n", "SELL", i, ", Optimal Put Chain:", "\n\n", optpzdf, "\n\n")
        except ValueError as err:
            print("no puts available for ", i, "\n\n")    

    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
dataframe_creator()
option_chain_generator()    
    
    

Stock TSLA is not within buying parameters
Stock  TSLA is within selling parameters. Dataframe:  

          Date Symbol   Close    EMA9   EMA20   EMA50       Volume
37  12/30/2022   TSLA  123.18  128.16  143.27  157.18  157,730,398

 SELL TSLA , Optimal Put Chain: 

    lastTradeDate  strike  lastPrice   bid   ask  volume  openInterest  \
20    12/30/2022   50.00       0.07  0.05  0.07  3422.0       20679.0   
21    12/30/2022   53.33       0.07  0.07  0.09   199.0        8178.0   
22    12/30/2022   55.00       0.08  0.07  0.10    65.0         320.0   
23    12/30/2022   60.00       0.13  0.12  0.13   991.0       28577.0   
24    12/30/2022   65.00       0.20  0.17  0.19    64.0        1257.0   
25    12/30/2022   66.67       0.22  0.19  0.22    69.0       31080.0   
26    12/30/2022   70.00       0.26  0.25  0.27  4774.0       14072.0   
27    12/30/2022   73.33       0.35  0.32  0.35   208.0        6086.0   
28    12/30/2022   75.00       0.37  0.36  0.39  1222.0        3041.0   
2

# 